# 🔁 Equilibrium Business Cycles with Persistence

This model builds on the insights of Lucas (1975) to simulate business cycles driven by **unanticipated monetary shocks** in an environment with **rational expectations** and **imperfect information**.

Crucially, it adds a mechanism for **persistence**: the effects of initial shocks don't die out immediately but ripple through time due to gradual adjustment of expectations and the influence of economic activity on **capital accumulation**.

We explore how a temporary monetary surprise can generate serially correlated deviations in output, prices, and the capital stock around their long-run trends.

# ⚙️ Model Structure (Log-Linear Deviations)

We model the economy using log-deviations ($x_t = \ln X_t - \ln \bar{X}$) from a non-stochastic steady state (normalized to zero).

1.  **Aggregate Demand (AD):** A simple quantity theory relationship.
    $$ y_t = m_t - p_t $$
    - $y_t$: Output deviation
    - $m_t$: Money supply deviation
    - $p_t$: Price level deviation

2.  **Aggregate Supply (AS - Lucas Supply + Capital):** Output deviates from its capital-determined potential ($\phi k_t$) due to price surprises.
    $$ y_t = \phi k_t + \gamma (p_t - p_t^e) $$
    - $k_t$: Capital stock deviation
    - $\phi$: Elasticity of potential output w.r.t. capital ($\approx \alpha$, capital's share)
    - $\gamma$: Sensitivity of output to price surprises (higher $\gamma$ -> flatter AS)
    - $p_t^e = E_{t-1}[p_t]$: Expected price level based on info at $t-1$.

3.  **Capital Accumulation (Simplified):** Net investment responds to output deviations (an accelerator effect), leading to changes in the capital stock deviation.
    $$ k_{t+1} = (1-\delta) k_t + s_y y_t $$
    - $\delta$: Depreciation rate.
    - $s_y$: Responsiveness of net investment to output deviations. *(This captures that higher output leads to higher saving/investment, building capital).*

4.  **Monetary Shock Process:** The money supply deviation follows an autoregressive process. A shock ($\epsilon_m$) hits at `shock_time`.
    $$ m_t = \rho m_{t-1} + \epsilon_{m,t} $$
    - $\rho$: Persistence of monetary deviations ($0 \le \rho < 1$).
    - $\epsilon_{m,t}$: Unanticipated monetary shock (non-zero only at `shock_time`).

5.  **Rational Expectations:** Agents know the model structure and parameters ($\phi, \gamma, \delta, s_y, \rho$) and form expectations $p_t^e$ based on information available at $t-1$ (including $k_t$ and $m_{t-1}$).
    $$ p_t^e = E_{t-1}[p_t] = E_{t-1}[m_t] - E_{t-1}[y_t] $$
    $$ p_t^e = (\rho m_{t-1}) - (\phi k_t + \gamma E_{t-1}[p_t - p_t^e]) $$
    $$ p_t^e = \rho m_{t-1} - \phi k_t $$
    *(Agents expect $p_t = p_t^e$, so the expected price surprise term is zero).*

**Equilibrium Determination:** In each period $t$, given the state ($k_t, m_{t-1}$) and the actual money supply $m_t$ (including any shock $\epsilon_{m,t}$), we solve the AD and AS equations simultaneously for the actual $p_t$ and $y_t$.

# 📊 Simulation and Interpretation

The simulation starts the economy in steady state ($k=0, m=0, y=0, p=0$). At `shock_time`, a one-time monetary shock ($\epsilon_m$) occurs. We then trace the dynamic response of the system over time.

**Observe:**
* **Initial Impact:** The shock ($\epsilon_m$) shifts AD. Because $p_t^e$ is based on past information, there's a price surprise ($p_t - p_t^e \neq 0$). Both $p_t$ and $y_t$ change.
* **Persistence:**
    * The monetary shock itself might persist if $\rho > 0$.
    * The initial output deviation ($y_t$) affects investment, causing the capital stock ($k_{t+1}$) to change.
    * Changes in $k$ shift potential output ($\phi k$) in the AS curve.
    * Changes in $k$ and the persistent effects in $m$ influence future price expectations ($p_t^e$).
    * These factors cause the effects of the initial shock to propagate and potentially oscillate over time as the economy returns to steady state.
* **Role of Parameters:**
    * $\gamma$: Governs the initial split between price and output effects (higher $\gamma$ -> more output effect).
    * $\rho$: Determines how long the monetary part of the shock lingers.
    * $\phi, s_y, \delta$: Control how strongly capital accumulation responds and feeds back into the cycle.

This setup captures the essence of Lucas's mechanism (monetary surprise -> real effects) combined with capital-based persistence.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass

def lucas_dynamic_model(
    gamma=1.5,
    phi=0.33,
    delta=0.08,
    s_y=0.1,
    rho=0.8,
    epsilon_m=0.05,
    T=50,
    shock_time=5
):
    T = int(T)
    shock_time = int(shock_time)
    if T < 10 or shock_time < 1 or shock_time >= T:
        print("Warning: T must be >= 10 and 1 <= shock_time < T.")
        return

    gamma = max(gamma, 1e-6)
    phi = np.clip(phi, 0.01, 0.99)
    delta = np.clip(delta, 0, 1)
    s_y = max(s_y, 0)
    rho = np.clip(rho, 0, 0.99)

    time = np.arange(T + 1)
    k = np.zeros(T + 1)
    m = np.zeros(T + 1)
    y = np.zeros(T)
    p = np.zeros(T)
    p_e = np.zeros(T)
    invest_net = np.zeros(T)

    for t in range(T):
        shock_now = epsilon_m if t == shock_time else 0.0
        m[t] = rho * m[t-1] + shock_now if t > 0 else shock_now
        p_e[t] = rho * m[t-1] - phi * k[t] if t > 0 else -phi * k[t]
        p[t] = (m[t] - phi * k[t] + gamma * p_e[t]) / (1.0 + gamma)
        y[t] = m[t] - p[t]
        invest_net[t] = s_y * y[t]
        k[t+1] = (1.0 - delta) * k[t] + invest_net[t]

    fig, axes = plt.subplots(2, 2, figsize=(14, 10), sharex=True)
    axes = axes.ravel()
    plot_time = time[:-1]

    axes[0].plot(time, m, label='Money Supply Dev (m)', color='blue', lw=2)
    axes[0].axvline(shock_time, color='red', linestyle='--', alpha=0.7, label=f'Shock (εₘ={epsilon_m:.3f})')
    axes[0].axhline(0, color='grey', linestyle=':', linewidth=0.5)
    axes[0].set_title("Monetary Shock Process")
    axes[0].set_ylabel("Log Deviation")
    axes[0].legend(fontsize='small')
    axes[0].grid(True, linestyle='--', alpha=0.6)

    axes[1].plot(plot_time, y, label='Output Dev (y)', color='green', lw=2)
    axes[1].plot(time, k, label='Capital Dev (k)', color='navy', lw=2, linestyle='--')
    axes[1].axhline(0, color='grey', linestyle=':', linewidth=0.5)
    axes[1].axvline(shock_time, color='red', linestyle='--', alpha=0.7)
    axes[1].set_title("Output and Capital Response")
    axes[1].set_ylabel("Log Deviation")
    axes[1].legend(fontsize='small')
    axes[1].grid(True, linestyle='--', alpha=0.6)

    axes[2].plot(plot_time, p, label='Actual Price Dev (p)', color='black', lw=2)
    axes[2].plot(plot_time, p_e, label='Expected Price Dev (pᵉ)', color='magenta', lw=2, linestyle=':')
    axes[2].axhline(0, color='grey', linestyle=':', linewidth=0.5)
    axes[2].axvline(shock_time, color='red', linestyle='--', alpha=0.7)
    axes[2].set_title("Price Level Response (Actual vs Expected)")
    axes[2].set_xlabel("Time (t)")
    axes[2].set_ylabel("Log Deviation")
    axes[2].legend(fontsize='small')
    axes[2].grid(True, linestyle='--', alpha=0.6)

    price_surprise = p - p_e
    ax3b = axes[3].twinx()
    line3a = axes[3].plot(plot_time, price_surprise, label='Price Surprise (p - pᵉ)', color='darkorange', lw=2)
    line3b = ax3b.plot(plot_time, invest_net, label='Net Investment Dev (i_net)', color='teal', lw=2, linestyle='--')
    axes[3].axhline(0, color='grey', linestyle=':', linewidth=0.5)
    axes[3].axvline(shock_time, color='red', linestyle='--', alpha=0.7)
    axes[3].set_title("Price Surprise and Investment Response")
    axes[3].set_xlabel("Time (t)")
    axes[3].set_ylabel("Price Surprise", color='darkorange')
    ax3b.set_ylabel("Net Investment", color='teal')
    axes[3].tick_params(axis='y', labelcolor='darkorange')
    ax3b.tick_params(axis='y', labelcolor='teal')
    lines = line3a + line3b
    labels = [l.get_label() for l in lines]
    axes[3].legend(lines, labels, loc='best', fontsize='small')
    axes[3].grid(True, linestyle='--', alpha=0.6)

    fig.suptitle(f"Dynamic Equilibrium Cycle Model (Lucas-Style with Capital)", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    peak_y_idx = np.argmax(np.abs(y))
    peak_y_val = y[peak_y_idx]
    peak_p_idx = np.argmax(np.abs(p))
    peak_p_val = p[peak_p_idx]
    peak_k_idx = np.argmax(np.abs(k[1:]))
    peak_k_val = k[peak_k_idx+1]

    summary_md = rf"""
### ⚙️ Simulation Summary

* **Peak Output Response (y):** {peak_y_val:.4f} at t = {peak_y_idx}  
* **Peak Price Response (p):** {peak_p_val:.4f} at t = {peak_p_idx}  
* **Peak Capital Response (k):** {peak_k_val:.4f} at t = {peak_k_idx+1}  

*This model shows how a monetary surprise ($\epsilon_m$ = {epsilon_m:.3f} at t = {shock_time}) generates persistent cyclical deviations in output, prices, and capital due to information lags ($\gamma$ = {gamma:.2f}) and capital accumulation ($s_y$ = {s_y:.2f}, $\delta$ = {delta:.2f}, $\phi$ = {phi:.2f}). Monetary persistence $\rho$ = {rho:.2f}.*
"""
    display(Markdown(summary_md))


# Create interactive UI
style = {'description_width': '160px'}
layout = Layout(width='95%')

interact(
    lucas_dynamic_model,
    gamma=FloatSlider(value=1.5, min=0.1, max=5.0, step=0.1, description='Supply Sensitivity (γ):', style=style, layout=layout),
    phi=FloatSlider(value=0.33, min=0.01, max=0.99, step=0.01, description='Output Elasticity (ϕ):', style=style, layout=layout),
    delta=FloatSlider(value=0.08, min=0.01, max=0.2, step=0.01, description='Depreciation Rate (δ):', style=style, layout=layout),
    s_y=FloatSlider(value=0.1, min=0.0, max=0.5, step=0.01, description='Investment Sensitivity (s_y):', style=style, layout=layout),
    rho=FloatSlider(value=0.8, min=0.0, max=0.99, step=0.01, description='Monetary Persistence (ρ):', style=style, layout=layout),
    epsilon_m=FloatSlider(value=0.05, min=-0.1, max=0.1, step=0.01, description='Shock Size (εₘ):', style=style, layout=layout),
    T=IntSlider(value=50, min=20, max=150, step=5, description='Time Horizon (T):', style=style, layout=layout),
    shock_time=IntSlider(value=5, min=1, max=20, step=1, description='Shock Period (t):', style=style, layout=layout)
)

interactive(children=(FloatSlider(value=1.5, description='Supply Sensitivity (γ):', layout=Layout(width='95%')…

<function __main__.lucas_dynamic_model(gamma=1.5, phi=0.33, delta=0.08, s_y=0.1, rho=0.8, epsilon_m=0.05, T=50, shock_time=5)>